<a href="https://colab.research.google.com/github/sinyoung0110/webcrawling/blob/main/%EC%9C%A0%ED%8A%9C%EB%B8%8C%EC%8D%B8%EB%84%A4%EC%9D%BC_crawlling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium

!apt-get update

!apt install chromium-chromedriver

!pip install chromedriver-autoinstaller

#최초1번 : 드라이브에 chromedriver 복사할 때 사용 하는 코드
#!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 29.9 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [70.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.ne

In [ ]:
!python --version

import selenium
print(selenium.__version__)

Python 3.11.11
4.30.0


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd
import os
import time
import re
import datetime


import chromedriver_autoinstaller

In [ ]:
chrome_path="/content/drive/MyDrive/Colab Notebooks/chromedriver"

In [ ]:
import sys
sys.path.insert(0,chrome_path)

chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # colab창을 띄울 수 없으므로 화면 없이 실행
chrome_options.add_argument('--no-sandbox') # 샌드박스 보완 비활성화(Colab 권한 문제 방지)
chrome_options.add_argument('--disable-dev-shm-usage') # 메모리 공간 부족 이슈 방지
chrome_options.add_argument('lang-ko_KR') # 브라우저 언어 한국어로 설정

#드라이버 설치
chromedriver_autoinstaller.install()

In [ ]:
# 페이지 아래로 스크롤하면서 새로운 콘텐츠가 로딩될 때까지 기다리고 더 이상 로딩되지 않을 때까지 반복하는 함수
def selenium_scroll_option(driver,scroll_count=5):
  SCROLL_PAUSE_SEC=2

  for i in range(scroll_count):
    # Removed the extra colon at the end of the JavaScript code
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    time.sleep(SCROLL_PAUSE_SEC)

In [ ]:
# 조회수, 좋아요 수 등의 문자열을 숫자로 변환하는 함수
def convert_count_to_number(count_str):
  if not count_str or count_str.lower()=='no views':
    return 0

  count_str=re.sub(r'[^0-9.KkMmBb만천억]','',count_str)

  if '만' in count_str:
    return float(count_str.replace('만',''))*10000
  elif '천' in count_str:
    return float(count_str.replace('천',''))*1000
  elif '억' in count_str:
    return float(count_str.replace('억',''))*100000000
  elif 'K' in count_str:
    return float(count_str.replace('K','').replace('k',''))*1000
  elif 'M' in count_str:
    return float(count_str.replace('M','').replace('m',''))*10000
  elif 'B' in count_str:
    return float(count_str.replace('B','').replace('b',''))*100000000
  else:
    return float(count_str) if count_str else 0

In [ ]:
# 날짜 문자열을 표준 형식으로 변환하는 함수

In [ ]:
def collect_youtube_videos(keyword,max_videos=10):
  base_url=f"https://www.youtube.com/results?search_query={keyword}"
  driver=webdriver.Chrome(options=chrome_options)
  driver.get(base_url)

  driver.implicitly_wait(5)

  selenium_scroll_option(driver,scroll_count=3)

  videos_data=[]
  video_elements =driver.find_elements(By.CSS_SELECTOR,"ytd-video-renderer")

  count=0
  for video in video_elements:
    if count >= max_videos:
      break

    try:
      title_element=video.find_element(By.CSS_SELECTOR,"#video-title")
      title=title_element.text

      link=title_element.get_attribute("href")
      channel_name=video.find_element(By.CSS_SELECTOR,"yt-simple-endpoint.style-scope.yt-formatted-string").text

      metadata=video.find_element(By.CSS_SELECTOR,"#metadata-line span")
      views=metadata[0].text if len(metadata)>1 else ""
      upload_data=metadata[1].text if len(metadata)>1 else ""

      driver.execute_script("window.open('');")
      driver.switch_to.window(driver.window_handles[1])
      driver.get(link)
      time.sleep(3)

      driver.close()
      driver.switch_to.window(driver.window_handles[0])

      views_count=convert_count_to_number(views)

      # 데이터 추가
      videos_data.append({
          "제목":title,
          "채널명":channel_name,
          "조회수":views_count,
          "업로드 날짜": upload_data,
          "영상 링크":link,
      })
      count+=1
    except Exception as e:
      print(f"영상 정보 수집 중 오류 발생 :{e}")
      continue
  driver.quit()
  return videos_data

In [ ]:
keyword=input("유튜브에서 검색할 키워드를 입력하세요 : ")

print(f"'{keyword}'키워드로 유튜브 영상 정보를 수집합니다...")
videos_data=collect_youtube_videos(keyword)

df=pd.DataFrame(videos_data)

print("\n수집된 영상 정보 : ")
print(df)

save_path=f"/content/drive/MyDrive/youtube_{keyword}_{datetime.datetime.now().strftime('%Y%m%d_%H%MS')}.csv"
df.to_csv(save_path,index=False, encoding='utf-8-sig')
print(f"\n 영상 정보가 다음 경로에 저장되었습니다.: {save_path}")

유튜브에서 검색할 키워드를 입력하세요 : cat
'cat'키워드로 유튜브 영상 정보를 수집합니다...
영상 정보 수집 중 오류 발생 :Message: no such element: Unable to locate element: {"method":"css selector","selector":"yt-simple-endpoint.style-scope.yt-formatted-string"}
  (Session info: chrome=135.0.7049.42); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5d454b67ce6a <unknown>
#1 0x5d454b12e640 <unknown>
#2 0x5d454b17fb5a <unknown>
#3 0x5d454b17fd51 <unknown>
#4 0x5d454b173606 <unknown>
#5 0x5d454b1a577d <unknown>
#6 0x5d454b1734fa <unknown>
#7 0x5d454b1a591e <unknown>
#8 0x5d454b1cb7b5 <unknown>
#9 0x5d454b1a5523 <unknown>
#10 0x5d454b171ef3 <unknown>
#11 0x5d454b172b51 <unknown>
#12 0x5d454b641d2b <unknown>
#13 0x5d454b645c01 <unknown>
#14 0x5d454b629512 <unknown>
#15 0x5d454b646774 <unknown>
#16 0x5d454b60d90f <unknown>
#17 0x5d454b66afb8 <unknown>
#18 0x5d454b66b196 <unknown>
#19 0x5d454b67bcb6 <unknown>
#20 0x131ed244ea